# Dynamic Programming

Using dynamic programming we can solve the RL problem when all environment dynamics are known and we have a finite state set.

Any environment that our agent can explore needs to be described using a subclass of `Environment`.

In [138]:
class Environment:
    def get_spaces(self):
        """
        Return the set of possible states
        """
        pass
    
    def get_possible_actions(self, state):
        """
        Returns the actions that can be taken from the given state
        """
        pass
    
    def execute_action(self, state, action):
        """
        Executes the action and changes the state
        """
        pass
    
    def reward(self, action, state, new_state):
        """
        Returns the reward for the given state change
        """
        pass

`DPAgent` can find a perfect policy given an `Environment`. It works using value iteration. Initially, a random policy is generated. After that, each time `agent.learn()` is executed, we perform:

* policy evaluation: The value for each state is updated, by using the values of neighbouring states
* policy improvement: For each state we choose the action that maximizes the expected value

In [87]:
from random import sample
from operator import itemgetter

class DPAgent:
    def __init__(self, env, discount_factor=1):
        self.env = env
        self.discount_factor = discount_factor
        
        states = self.states = env.get_spaces()
        self.values = { state: 0 for state in states }
        self.policy = { state: set(sample(env.get_possible_actions(state), 1)) for state in states }
        
    def learn(self):
        self._evaluate_policy()
        self._improve_policy()
        
    def _evaluate_policy(self):
        values = self.values.copy()
        
        for state in self.states:
            value = self.values[state]
            actions = self.policy[state]
            change = 0.
            
            for action in actions:
                new_state = self.env.execute_action(state, action)
                next_value = self.values[new_state]
                reward = self.env.reward(action, state, new_state)
                
                change += reward + self.discount_factor * next_value
            
            values[state] = change / len(actions)

        self.values = values
    
    def _improve_policy(self):
        for state in self.states:
            action = self.policy[state]
            possible_actions = {}

            for possible_action in self.env.get_possible_actions(state):
                new_state = self.env.execute_action(state, possible_action)
                
                value = self.values[new_state]
                reward = self.env.reward(action, state, new_state)
                
                possible_actions[possible_action] = reward + self.discount_factor * value
            
            max_value = max(possible_actions.values())            
            self.policy[state] = set([action for action, value in possible_actions.items() if value == max_value])

## GridWorld

GridWorld is on of the standard examples for dynamic programming. Our agent can move in a 2d matrix. The rewards are defined in a matrix of the same size. There environment is totally deterministic.

In [3]:
from itertools import product
import numpy as np

class GridWorld(Environment):
    def __init__(self, reward_matrix):
        self.reward_matrix = reward_matrix
        
        n, m = reward_matrix.shape
        self.states = list(product(range(n), range(m)))
        
        self.max_down = n - 1
        self.max_right = m - 1
        
        self._init_actions()
        
    def _init_actions(self):
        self.UP = "U"
        self.DOWN = "D"
        self.LEFT = "L"
        self.RIGHT = "R"
        
        self.actions = [self.UP, self.DOWN, self.LEFT, self.RIGHT]
        
    def get_spaces(self):
        return self.states
    
    def get_possible_actions(self, state):
        i, j = state
        
        actions = []
        
        if i > 0:
            actions.append(self.UP)
            
        if i < self.max_down:
            actions.append(self.DOWN)
        
        if j > 0:
            actions.append(self.LEFT)
        
        if j < self.max_right:
            actions.append(self.RIGHT)
            
        return actions
    
    def execute_action(self, state, action):
        i, j = state
        
        if action == self.UP:
            i -= 1
            
        if action == self.DOWN:
            i += 1
            
        if action == self.LEFT:
            j -= 1
            
        if action == self.RIGHT:
            j += 1
            
        i = max(0, min(i, self.max_down))
        j = max(0, min(j, self.max_right))
        
        return i, j
    
    def reward(self, action, state, new_state):
        return self.reward_matrix[state]

Some helper functions for visualizing the policy/values/rewards of GridWorld.

In [139]:
def draw_policy(agent, n, m):
    lookup = {
        "U": "^",
        "D": "v",
        "L": "<",
        "R": ">"
    }
    
    lookup = {
        "U": u"↑",
        "D": u"↓",
        "L": u"←",
        "R": u"→"
    }
    
    vals = sorted(agent.policy.items())
    actions = ["".join([lookup[act] for act in val]) for _, val in vals]
    
    print "Policy"
    draw_matrix(np.array(actions).reshape((n, m)))
    print

def draw_values(agent, n, m):
    vals = sorted(agent.values.items())
    vals = ["%.2f" % val for _, val in vals]
    
    print "Values"
    draw_matrix(np.array(vals).reshape((n, m)))
    print
    
from math import ceil, floor

def lrjust(s, size):
    missing = size - len(s)
    
    if missing > 0:
        left = int(ceil(missing / 2.))
        right = int(floor(missing / 2.))
        s = " " * left + s + " " * right
    
    return s

def draw_matrix(data):    
    result = ""
    rows = []
    
    for row in data:
        row_result = []
        
        for cell in row:
            row_result.append(lrjust("%s" % cell, 6))
            
        rows.append("|".join(row_result))
        
    print ("\n" + "-" * len(rows[0]) + "\n").join(rows)

### Test cases

In this simple test there's only one cell with a reward, so what we're really looking for is the shortest path to that cell.

If a cell of the policy matrix contains several action, then this means that we can take any of them.

In [141]:
R = np.array([
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0]
    ])

print "Reward"
draw_matrix(R)
print

grid_world = GridWorld(R)
agent = DPAgent(grid_world, discount_factor=0.5)

for k in range(10):
    agent.learn()
    
draw_policy(agent, 3, 4)
draw_values(agent, 3, 4)

Reward
   0  |   0  |   0  |   0  
---------------------------
   0  |   0  |   1  |   0  
---------------------------
   0  |   0  |   0  |   0  

Policy
  →↓  |  →↓  |   ↓  |  ↓←  
---------------------------
   →  |   →  | →↑↓← |   ←  
---------------------------
  →↑  |  →↑  |   ↑  |  ↑←  

Values
 0.17 | 0.33 | 0.67 | 0.33 
---------------------------
 0.33 | 0.67 | 1.33 | 0.67 
---------------------------
 0.17 | 0.33 | 0.67 | 0.33 



In this test case there's a single path to a reward.

In [137]:
R2 = np.array([
        [0, 0, 0, 0],
        [0, -1, -1, -1],
        [0, 0, 0, 0],
        [-1, -1, -1, 0],
        [1, 0, 0, 0]
    ])

print "Reward"
draw_matrix(R2)
print

grid_world = GridWorld(R2)
agent = DPAgent(grid_world, discount_factor=0.5)

for k in range(50):
    agent.learn()
    
draw_policy(agent, 5, 4)
draw_values(agent, 5, 4)

Reward
   0  |   0  |   0  |   0  
---------------------------
   0  |  -1  |  -1  |  -1  
---------------------------
   0  |   0  |   0  |   0  
---------------------------
  -1  |  -1  |  -1  |   0  
---------------------------
   1  |   0  |   0  |   0  

Policy
   ↓  |   ←  |   ←  |   ←  
---------------------------
   ↓  |   ↓  |   ↓  |   ↓  
---------------------------
   →  |   →  |   →  |   ↓  
---------------------------
   ↓  |   ↓  |   ↓  |   ↓  
---------------------------
   →  |   ←  |   ←  |   ←  

Values
 0.00 | 0.00 | 0.00 | 0.00 
---------------------------
 0.00 | -0.99| -0.99| -0.98
---------------------------
 0.01 | 0.01 | 0.02 | 0.04 
---------------------------
 -0.33| -0.67| -0.83| 0.08 
---------------------------
 1.33 | 0.67 | 0.33 | 0.17 

